## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [SETTINGS.md](SETTINGS.md) file.

## 📋 Table of Contents

This notebook assists in creating an Azure AI Search Index, covering the following sections:

1. [**Define Field Types**](#define-field-types): Outlines the process of defining the structure and behavior of an index using various field types.

2. [**Configuring Vector Search**](#configuring-vector-search): Discusses the setup of algorithms and profiles for handling vector-based queries.

3. [**Configuring Semantic Search**](#configuring-semantic-search): Explores how to enhance search capabilities by leveraging advanced AI models.

4. [**Create or Update Index**](#create-or-update-index): Details the steps to create a new index or update an existing one.

For additional information, refer to the following resources:
- [Azure AI Search Documentation](https://learn.microsoft.com/en-us/azure/search/)

In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-aoai-faq


In [2]:
from dotenv import load_dotenv
from openai import AzureOpenAI

from utils.ml_logging import get_logger

# Load environment variables from .env file
load_dotenv()
# Set up logger
logger = get_logger()

In [3]:
from src.aoai.azure_openai import AzureOpenAIManager

In [4]:
import requests
import os
import logging
import traceback

# Setup logger
logger = logging.getLogger(__name__)


In [5]:
# Usage
# Create an instance of the client
azure_openai_client = AzureOpenAIManager()

In [6]:
from typing import Dict, Optional
from requests import Response

def call_azure_openai_chat_completions_api(deployment_id: str, 
                                           method: str, 
                                           body: dict = None, api_version: str = "2023-11-01"):
    """
    Calls the Azure OpenAI API with the given parameters.

    :param deployment_id: The ID of the deployment to access.
    :param method: The HTTP method to use ("get" or "post").
    :param body: The body of the request for "post" method. Defaults to None.
    :param api_version: The API version to use. Defaults to "2023-11-01".

    :return: The status code and response from the API call, along with rate limit headers.
    """
    if method.lower() not in ["get", "post"]:
        logger.error("Invalid HTTP method. Expected 'get' or 'post'.")
        return None, None, {}

    url = f"{azure_openai_client.azure_endpoint}/openai/deployments/{deployment_id}/chat/completions?api-version={api_version}"
    headers = {
        "Content-Type": "application/json",
        "api-key": azure_openai_client.api_key
    }

    with requests.Session() as session:
        session.headers.update(headers)

        try:
            if method.lower() == "get":
                response = session.get(url)
            else:  # method.lower() == "post"
                response = session.post(url, json=body)
            response.raise_for_status()  # Raises HTTPError for bad responses
        except requests.HTTPError as http_err:
            logger.error(f"HTTP error occurred: {http_err}")
            return response.status_code, http_err.response.json(), {}
        except Exception as err:
            logger.error(f"An error occurred: {err}")
            return None, None, {}

    # Extract rate limit headers and usage details
    rate_limit_headers = extract_rate_limit_and_usage_info(response)
    return response.status_code, response.json(), rate_limit_headers

def extract_rate_limit_and_usage_info(response: Response) -> Dict[str, Optional[int]]:
    """
    Extracts rate limiting information from the Azure Open API response headers and usage information from the payload.

    :param response: The response object returned by a requests call.
    :return: A dictionary containing the remaining requests, remaining tokens, and usage information 
             including prompt tokens, completion tokens, and total tokens.
    """
    headers = response.headers
    usage = response.json().get('usage', {})
    return {
        'remaining-requests': headers.get('x-ratelimit-remaining-requests'),
        'remaining-tokens': headers.get('x-ratelimit-remaining-tokens'),
        'prompt-tokens': usage.get('prompt_tokens'),
        'completion_tokens': usage.get('completion_tokens'),
        'total_tokens': usage.get('total_tokens'),
    }

In [7]:
os.getenv("AZURE_AOAI_CHAT_MODEL_DEPLOYMENT_ID")

'foundational-canadaeast-gpt4'

In [8]:
azure_openai_client.chat_model_name

'foundational-canadaeast-gpt4'

In [9]:
import time

body = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"},
        {"role": "assistant", "content": "Yes, other Azure AI services also support customer managed keys."},
        {"role": "user", "content": "Can you tell me more about these services?"},
        {"role": "assistant", "content": "Sure, Azure AI services include Azure Cognitive Services, Azure Machine Learning, and more."},
        {"role": "user", "content": "What is Azure Cognitive Services?"},
        {"role": "assistant", "content": "Azure Cognitive Services is a collection of APIs and services for building intelligent applications."},
        {"role": "user", "content": "What is Azure Machine Learning?"},
        {"role": "assistant", "content": "Azure Machine Learning is a cloud-based service for building, training, and deploying machine learning models."},
        {"role": "user", "content": "Thank you for the information."},
        {"role": "assistant", "content": "You're welcome! If you have any other questions, feel free to ask."},
        {"role": "user", "content": "What other services does Azure offer?"},
        {"role": "assistant", "content": "Azure offers a wide range of services including computing, analytics, storage, and networking."},
        {"role": "user", "content": "Can you tell me more about Azure's computing services?"},
        {"role": "assistant", "content": "Azure's computing services include virtual machines, container services, and serverless computing."},
        {"role": "user", "content": "What is serverless computing?"},
        {"role": "assistant", "content": "Serverless computing is a cloud computing model where the cloud provider automatically manages the provisioning and scaling of servers."},
        {"role": "user", "content": "That's interesting. Thank you for the information."},
        {"role": "assistant", "content": "You're welcome! If you have any other questions, feel free to ask."},
     ]
    }
# ,
#     "max_tokens": 150,
#     "n": 1,
#     "stream": True,

import time

status_code, response, rate_limit_info = call_azure_openai_chat_completions_api(
    deployment_id=azure_openai_client.completion_model_name,
    method='post',
    body=body,
    api_version='2023-05-15'
)

# Print the status code, response, and rate limit info
print("Status Code:", status_code)
print("Response:", response)
print("Rate Limit Info:", rate_limit_info)

Status Code: 200
Response: {'id': 'chatcmpl-8l8uTP8PSUS1iQFgRhpD0lxSC5JPz', 'object': 'chat.completion', 'created': 1706246357, 'model': 'gpt-4', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "You're welcome! If you have any more questions or need further information on any topic, don't hesitate to ask. I'm here to help!"}}], 'usage': {'prompt_tokens': 333, 'completion_tokens': 30, 'total_tokens': 363}, 'system_fingerprint': 'fp_6d044fb900'}
Rate Limit Info: {'remaining-requests': '9', 'remaining-tokens': '9013', 'prompt-tokens': 333, 'completion_tokens': 30, 'total_tokens': 363}


In [16]:
status_code, response, rate_limit_info = call_azure_openai_chat_completions_api(
        deployment_id=azure_openai_client.completion_model_name,
        method='post',
        body=body,
        api_version='2023-05-15'
    )

HELLO ------------------
{'Cache-Control': 'no-cache, must-revalidate', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/event-stream', 'access-control-allow-origin': '*', 'apim-request-id': 'ed5e43ca-d161-4eab-9fb9-c9b8110a958b', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'Canada East', 'x-ratelimit-remaining-requests': '9', 'x-ratelimit-remaining-tokens': '9006', 'x-accel-buffering': 'no', 'x-request-id': 'e959c574-57a3-40eb-9c03-10dcf182de9c', 'x-ms-client-request-id': 'ed5e43ca-d161-4eab-9fb9-c9b8110a958b', 'azureml-model-session': 'd008-20240109104201', 'Date': 'Thu, 25 Jan 2024 06:02:24 GMT'}
HELLO ------------------


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
rate_limit_info

{'limit-requests': None,
 'limit-tokens': None,
 'remaining-requests': '9',
 'remaining-tokens': '9013',
 'reset-requests': None,
 'reset-tokens': None}